In [23]:
# Import the Python SDK
import google.generativeai as genai
from dotenv import load_dotenv
import os
import pandas as pd
from pyhtml2pdf import converter


In [3]:
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI-API-KEY")

In [4]:
genai.configure(api_key=GEMINI_API_KEY)

In [5]:
model = genai.GenerativeModel('gemini-1.5-flash')

In [6]:
reviews = pd.read_csv("../Scrapedreviews/Review_Data_3.csv")

In [7]:
reviews.head(2)

,Unnamed: 0,title,rating,content
0,0,Classy look and feel,5,"Really nice texture and feel, chose the perfec..."
1,1,feels good on wool side and other side have be...,4,the wool side feels softs and good to place yo...


In [8]:
reviews = reviews.drop(reviews.columns[0], axis = 1)
reviews.head()

,title,rating,content
0,Classy look and feel,5,"Really nice texture and feel, chose the perfec..."
1,feels good on wool side and other side have be...,4,the wool side feels softs and good to place yo...
2,Good quality,5,Good quality nice crafts ship and nice tying s...
3,Good for table use,5,Still use today mouse & keyboard get good grip
4,Great quality,5,Great quality


In [9]:
reviews.rename(columns = {'content': 'body'}, inplace = True)
reviews.head()

,title,rating,body
0,Classy look and feel,5,"Really nice texture and feel, chose the perfec..."
1,feels good on wool side and other side have be...,4,the wool side feels softs and good to place yo...
2,Good quality,5,Good quality nice crafts ship and nice tying s...
3,Good for table use,5,Still use today mouse & keyboard get good grip
4,Great quality,5,Great quality


In [10]:
reviews

,title,rating,body
0,Classy look and feel,5,"Really nice texture and feel, chose the perfec..."
1,feels good on wool side and other side have be...,4,the wool side feels softs and good to place yo...
2,Good quality,5,Good quality nice crafts ship and nice tying s...
3,Good for table use,5,Still use today mouse & keyboard get good grip
4,Great quality,5,Great quality
5,This is not very smooth surface,3,I am not happy with the quality and it is too ...
6,gives your desk a nice and clean look but attr...,4,The only issue is that cleaning the dust from ...
7,Size too small,1,Product is fine but too small for a laptop des...
8,Good,4,Good
9,Very good,5,To my liking and satisfaction.


In [11]:
rvwStruct = {"reviews": []}

for i in range(len(reviews['title'])):
    inst = reviews.iloc[i]
    rvw = {"rating": inst['rating'], "title": inst['title'], "body": inst['body']}
    rvwStruct['reviews'].append(rvw)

In [12]:
rvwStruct['reviews'][:10]

[{'rating': 5,
  'title': 'Classy look and feel',
  'body': 'Really nice texture and feel, chose the perfect colour and thats it.'},
 {'rating': 4,
  'title': 'feels good on wool side and other side have bend marks on it.',
  'body': 'the wool side feels softs and good to place your hand on it.but it is definitely rough for mouse . good for day to day use but strictly not for gamers and it really cannot clean on wool side'},
 {'rating': 5,
  'title': 'Good quality',
  'body': 'Good quality nice crafts ship and nice tying strap. Faux leather is very smooth.'},
 {'rating': 5,
  'title': 'Good for table use',
  'body': 'Still use today mouse & keyboard get good grip'},
 {'rating': 5, 'title': 'Great quality', 'body': 'Great quality'},
 {'rating': 3,
  'title': 'This is not very smooth surface',
  'body': 'I am not happy with the quality and it is too expensive I would like to return. Please cancel this order'},
 {'rating': 4,
  'title': 'gives your desk a nice and clean look but attracts 

In [13]:
prompt1 = """You are an advanced language model tasked with analyzing customer reviews for a specific product. The reviews include a rating out of 5, a title, and the main body of the review. Your objective is to extract meaningful insights that will help a potential buyer make an informed decision about whether to purchase the product. Your analysis should provide:

Overall Sentiment Analysis:

Summarize the general sentiment of the reviews (positive, negative, or mixed).
Highlight any patterns or trends in the ratings (e.g., most reviews are 4 stars, consistent 5-star ratings, frequent 1-star complaints).
Pros:

Identify and list the most common positive aspects mentioned by reviewers.
Provide specific examples of features, qualities, or experiences that customers appreciated.
Cons:

Identify and list the most common negative aspects mentioned by reviewers.
Highlight any recurring issues, complaints, or dissatisfaction points raised by multiple reviewers.
Key Insights:

Extract any additional insights that do not fall strictly under pros or cons but are important for understanding the product (e.g., variations in product performance, unique use cases, or specific customer needs addressed).
Note any discrepancies between high ratings and negative comments or low ratings with positive comments.
Decision-Making Summary:

Provide a final summary that encapsulates the overall crux of the reviews.
Offer a balanced view that weighs the pros and cons to assist the potential buyer in making an informed decision.
Input Format:

Rating: [Rating out of 5]
Title: [Title of the review]
Body: [Body of the review]
Output Format:

Overall Sentiment Analysis:
[Summary of general sentiment and rating trends]
Pros:
[List of pros with examples]
Cons:
[List of cons with examples]
Key Insights:
[Additional important insights]
Decision-Making Summary:
[Balanced summary to aid in decision-making]
Here are the reviews:  
"""

In [14]:
type(prompt1)

str

In [15]:
str(rvwStruct)

'{\'reviews\': [{\'rating\': 5, \'title\': \'Classy look and feel\', \'body\': \'Really nice texture and feel, chose the perfect colour and thats it.\'}, {\'rating\': 4, \'title\': \'feels good on wool side and other side have bend marks on it.\', \'body\': \'the wool side feels softs and good to place your hand on it.but it is definitely rough for mouse . good for day to day use but strictly not for gamers and it really cannot clean on wool side\'}, {\'rating\': 5, \'title\': \'Good quality\', \'body\': \'Good quality nice crafts ship and nice tying strap. Faux leather is very smooth.\'}, {\'rating\': 5, \'title\': \'Good for table use\', \'body\': \'Still use today mouse & keyboard get good grip\'}, {\'rating\': 5, \'title\': \'Great quality\', \'body\': \'Great quality\'}, {\'rating\': 3, \'title\': \'This is not very smooth surface\', \'body\': \'I am not happy with the quality and it is too expensive I would like to return. Please cancel this order\'}, {\'rating\': 4, \'title\': \

In [16]:
rvwAnalysis = prompt1 + str(rvwStruct)
response = model.generate_content(rvwAnalysis)

In [17]:
print(response.text)

## Desk Mat Review Analysis

**Overall Sentiment Analysis:**

The overall sentiment of the reviews is **mixed**. While there are a significant number of positive reviews (5-star ratings), there are also many negative reviews (1-star ratings) highlighting major concerns about the product. The rating distribution shows a trend towards polarized opinions with a considerable number of reviews at the extremes (1 star and 5 stars), while the middle ground (3 and 4 stars) receives fewer reviews.

**Pros:**

* **Good Quality and Feel:** Several reviewers praised the quality of the materials, particularly the leather side and the felt side's softness. 
    * Example: "Really nice texture and feel, chose the perfect colour and thats it."
    * Example: "Good quality nice crafts ship and nice tying strap. Faux leather is very smooth."
* **Aesthetically Pleasing:** The desk mat's appearance is a common positive point, with reviewers appreciating its classy and luxurious look.
    * Example: "Feels

In [18]:
promptn = r"""
            Convert the following textual report into well-structured LaTeX code.
            Ensure the document includes appropriate sections, subsections, and any necessary formatting such as lists, tables, figures, and references. 
            keep the following instructions in mind while making the LaTeX code:
            *Use the \textbf{text} command for bold text.
            *Use a standard apostrophe (') for possessives and contractions.
            For quotation marks:
            Use ` (two backticks) for opening quotes.
            Use '' (two single quotes) for closing quotes.
            Avoid using backslashes before apostrophes or quotes unless invoking special characters.
            The output should be a complete LaTeX document
            ready for compilation.
            *Ensure that the necessary packages are included at the beginning of the document, such as \usepackage{amsmath}, \usepackage{graphicx}, \usepackage{enumitem}, and \usepackage{url} if their features are being utilized.
            *Follow LaTeX conventions for formatting, including consistent spacing, indentation for readability, and proper use of sections and environments.
            *Avoid inserting unnecessary newline characters or special characters like \n that are not valid in LaTeX.
            * After creating the code double check it for any syntax errors.
            The report is: \n
"""

In [19]:
prompt2 = r""" 
        Convert the following report into a well-structured HTML page. 
        Ensure that the content remains unchanged and that all sections and subsections are preserved. 
        Use appropriate HTML tags to represent the structure of the document, including headers for sections, 
        paragraphs for text, and lists where applicable. Maintain the hierarchy of the sections and subsections as they appear in the original report.
        DO NOT USE '\n'
        Here is the report: \n
"""

In [20]:
final_report = prompt2 + response.text
response_rep = model.generate_content(final_report)

In [21]:
print(type(response_rep.text))

<class 'str'>


In [24]:
path = os.path.abspath('index.html')
converter.convert(f'{path}', 'sample.pdf')